In [ ]:
import scipy.io as scipy
from scipy import stats
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, make_scorer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
import pickle
import time
import pandas as pd
import subprocess
import joblib
import matplotlib.pyplot as plt
import json

In [ ]:
def run_bash_command(command):
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        print("Output:", output.decode('utf-8'))
    except subprocess.CalledProcessError as e:
        print("Error:", e.output.decode('utf-8'))
        return None

In [ ]:
class EvaluateSVM(object):

  SOURCEPATH = '..'
  OFFSET = 1e-64
  MAXITER = 1000
  RANDOMSTATE = 42

  KERNEL = 'poly'
  GAMMA = 10
  DEGREE = 3
  COEF0 = 10
  C = 1

  def __init__(self, method, train_dataset, test_dataset):
    self.train_dataset = train_dataset
    self.test_dataset = test_dataset
    self.X_train = None
    self.X_test = None
    self.y_train = None
    self.y_test = None
    self.model = None
    self.method = method

  def load_features(self, dataset):
    features_path = f"{self.SOURCEPATH}/data/gmm/{dataset.replace('.npy','')}"
    features = np.load(f'{features_path}/input_features.npy')
    labels = np.load(f'{features_path}/labels.npy')

    return features, labels

  def load_data(self):
    self.X_train, self.y_train = self.load_features(self.train_dataset)
    self.X_test, self.y_test = self.load_features(self.test_dataset)

    print(f"X_train shape: {self.X_train.shape}")
    print(f"y_train shape: {self.y_train.shape}")

    print(f"X_test shape: {self.X_test.shape}")
    print(f"y_test shape: {self.y_test.shape}")


  def load_model(self):

    try:
      model_path = f"{self.SOURCEPATH}/models/cross_dataset/{self.train_dataset.replace('.npy','')}/svm"
      self.model = joblib.load(f'{model_path}/svm_model.pkl')
      print(f'Model loaded from {model_path}')
    except:
      print("Model Not Found!")
      print('Training Model!')
      self.train_model()

  def train_model(self):
    # Train the SVM model
    self.model = SVC(
        kernel=self.KERNEL,
        C=self.C,
        gamma=self.GAMMA
    )
    self.model.fit(self.X_train, self.y_train)

    # Save the model
    model_path = f"{self.SOURCEPATH}/models/cross_dataset/{self.train_dataset.replace('.npy','')}/svm"
    try:
        run_bash_command(f"mkdir -p {model_path}")
    except:
        print("Folder already exists!")

    print(f"Saving model on {model_path}")
    joblib.dump(self.model, f'{model_path}/svm_model.pkl')

  def calc_metrics(self, metrics_features, metrics_labels):
     # Predict the training labels
    predictions = self.model.predict(metrics_features)

    # Calculate and display metrics
    accuracy = accuracy_score(metrics_labels, predictions)
    precision = precision_score(metrics_labels, predictions, average=self.method)
    recall = recall_score(metrics_labels, predictions, average=self.method)
    f1 = f1_score(metrics_labels, predictions, average=self.method)
    balanced_accuracy = balanced_accuracy_score(metrics_labels, predictions)

    metrics_dict = {"Accuracy":accuracy,
                 "Precision":precision,
                 "Recall":recall,
                 "F1-Score":f1,
                 "Balanced Accuracy":balanced_accuracy
                 }
    # Compute and plot the confusion matrix
    cm = confusion_matrix(metrics_labels, predictions)

    return metrics_dict, cm

  def save_metrics(self):
    train_metrics, train_cm = self.calc_metrics(self.X_train, self.y_train)
    test_metrics, test_cm = self.calc_metrics(self.X_test, self.y_test)

    disp = ConfusionMatrixDisplay(confusion_matrix=train_cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix for Training Phase")
    plt.show()

    disp = ConfusionMatrixDisplay(confusion_matrix=test_cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix for Testing Phase")
    plt.show()

    print(train_metrics)

    print(test_metrics)

    metrics_path = f"{self.SOURCEPATH}/metrics/cross_dataset/{self.train_dataset.replace('.npy','')}"

    try:
      run_bash_command(f"mkdir -p {metrics_path}")
    except:
      print("Folder already exist!")

    np.save(f"{metrics_path}/svm_train_cm.npy", train_cm)
    np.save(f"{metrics_path}/svm_test_cm.npy", test_cm)

    with open(f"{metrics_path}/svm_metrics_train.json", "w") as outfile:
        json.dump(train_metrics, outfile, indent=4)

    with open(f"{metrics_path}/svm_metrics_test.json", "w") as outfile:
        json.dump(test_metrics, outfile, indent=4)

  def evaluate_model(self):
    self.load_data()
    self.load_model()
    self.save_metrics()

In [ ]:
method='weighted'

train_dataset = 'window_4000_overlap_1000_hzdr_norm.npy'
test_dataset = 'window_4000_overlap_1000_tud_norm.npy'

EVAL = EvaluateSVM(method, train_dataset, test_dataset)
EVAL.evaluate_model()

In [ ]:
train_dataset = 'window_4000_overlap_1000_tud_norm.npy'
test_dataset = 'window_4000_overlap_1000_hzdr_norm.npy'


EVAL = EvaluateSVM(method, train_dataset, test_dataset)
EVAL.evaluate_model()